In [ ]:
import numpy as np # linear algebra
import pandas as pd #veri işleme, CSV dosyası G / Ç

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
import os

In [ ]:
#input içerisindekileri listeledik.
print(os.listdir("../input"))

In [ ]:
#Eğitim dosyasını okuyup train oluşturduk.
#Eğitim verileri çok büyük bu yüzden hafızadan tasarruf etmek için veri türlerini belirttik.
train = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [ ]:
#Ortalama değerlere yuvarlanmış acoustic_data grafiği oluşturduk.
grafik = train['acoustic_data'][:100]
grafik_smooth = grafik.rolling(10).mean()

plt.plot(grafik)
plt.plot(grafik_smooth)
plt.show()

In [ ]:
# Verilen değişken ile hedef değişken arasındaki ilişkiyi görmek için grafik ve her iki özelliğe dayalı olarak çizim yapma işlevi gerçekleştirdik.
acoustic_data_train = train['acoustic_data'].values[::100]
time_to_failure_train = train['time_to_failure'].values[::100]
fig, ax1 = plt.subplots(figsize=(18, 8))
plt.title("Acoustic_data ve time_to_failure Eğilimleri")
plt.plot(acoustic_data_train, color='crimson')
ax1.set_ylabel('acoustic_data', color='crimson')
plt.legend(['acoustic_data'])
ax2 = ax1.twinx()
plt.plot(time_to_failure_train, color='b')
ax2.set_ylabel('time_to_failure', color='b')
plt.legend(['time_to_failure'], loc=(0.9, 0.9))
plt.grid(False)
#eski çerçeveyi sildik
del acoustic_data_train
del time_to_failure_train

In [ ]:
# Pandas bize tüm ondalık sayıları göstermez
pd.options.display.precision = 10
train.head()

In [ ]:
# Train verileri için özellik oluşturma
# Basit türetilmiş özelliklere sahip bir eğitim dosyası oluşturduk


rows = 150_000 #150_000
segments = int(np.floor(train.shape[0] / rows))

X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ort', 'std', 'max', 'min'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values[-1]
    
    y_train.loc[segment, 'time_to_failure'] = y
    
    X_train.loc[segment, 'ort'] = x.mean()
    X_train.loc[segment, 'std'] = x.std()
    X_train.loc[segment, 'max'] = x.max()
    X_train.loc[segment, 'min'] = x.min()

In [ ]:
X_train.head()

In [ ]:
#Tüm değerlerin eşit olarak katkıda bulunmasını sağlamak için değerlerin aynı birime getirilmesi gerektiğinden ölçeklendirdik.
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)


In [ ]:
X_train_scaled

In [ ]:
#Catboost 
c = CatBoostRegressor(loss_function='MAE')
c.fit(X_train_scaled, y_train.values.flatten(), silent=True)
y_pred = c.predict(X_train_scaled)

In [ ]:
#Tahmin edilen ve Gerçek değerlerin görselleştirilmesi
plt.figure(figsize=(8, 8))
plt.scatter(y_train.values.flatten(), y_pred, c='crimson')
plt.xlim(0, 20)
plt.ylim(0, 20)
plt.xlabel('Gerçek', fontsize=10)
plt.ylabel('Tahmin Edilen', fontsize=10)
plt.plot([(0, 0), (15, 15)], [(0, 0), (15, 15)])
plt.show()

In [ ]:
#Ortalama Mutlak Hata (iki sürekli değişken arasındaki farkın ölçüsüdür)
#Düşük değerlere sahip tahminleyiciler daha iyi performans gösterir.
score = mean_absolute_error(y_train.values.flatten(), y_pred)
print(f'Score: {score:0.3f}')

In [ ]:
# submission(gönderim) dosyasını input dizininden okuma
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')

In [ ]:
# test verisini X_train ile aynı sütunları kullanarak oluşturduk.
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [ ]:
# Test verileri için özellik oluşturduk.
for seg_id in X_test.index:
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ort'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()

In [ ]:
# Submission(gönderim) dosyasını tahminlerle doldurduk.
X_test_scaled = scaler.transform(X_test)
submission['time_to_failure'] = c.predict(X_test_scaled)
# csv dosyasına yazdık
submission.to_csv('submission.csv')
submission.head()

In [ ]:
#Catboost ile tahmin ve gerçek grafiği oluşturduk.
plt.figure(figsize=(18, 8))
plt.plot(y_train, color='g', label='y_train')
plt.legend(['y_train'], loc=(0.88, 0.95))
plt.plot(y_pred, color='b', label='cat')
plt.title('Catboost');
plt.suptitle('Tahminler vs Gerçek');
